In [2]:
import pandas as pd
import pandas.io.sql as pd_sql
from sodapy import Socrata
import requests 
import time
import sqlite3 as sq
import sqlalchemy 
from bs4 import BeautifulSoup
import json
import numpy as np
from tqdm import tqdm

### DCFS referrals

In [ ]:
client_dcfs = Socrata('data.lacounty.gov','7pTgt6f2oTY53aDI1jXNJoNZD')
offset_temp = 0
dcfs_df=pd.DataFrame()

pbar = tqdm(total=64)

while True:
    results = client_dcfs.get("8vmx-hhtu", limit=5000, offset=offset_temp)
    dcfs_df_temp = pd.DataFrame.from_records(results)
    if dcfs_df_temp.empty == True:
        break
    dcfs_df = dcfs_df.append(dcfs_df_temp)
    offset_temp+=5000
    time.sleep(1)
    pbar.update(1)
pbar.close()

In [ ]:
dcfs_df['location'] = dcfs_df['location'].astype('str')
conn = sq.connect("testReferral.db")
cur = conn.cursor()
pd_sql.to_sql(dcfs_df, 't_dcfs', conn, if_exists='replace', index=False)
conn.close()

### Get LA City zip codes

In [ ]:
# Get all zip codes in Los Angeles City (http://geohub.lacity.org/datasets/875d540d71e64b8696cc368865c2b640_0)
url="https://services5.arcgis.com/7nsPwEMP38bSkCjy/arcgis/rest/services/Intersect of City Boundary and Zip Codes (LA County)/FeatureServer/0/query?where=1%3D1&outFields=ZIPCODE&outSR=4326&f=json"
r = requests.get(url)
data = json.loads(r.text)
lacity_zip = list()
for i in range(len(data['features'])):
    try:
        zipcode = data['features'][i]['attributes']['ZIPCODE']
        lacity_zip.append((i+1,zipcode))
    except:
        continue
        

In [ ]:
lacity_zip_df = pd.DataFrame(lacity_zip)
lacity_zip_df.columns=['primary_key','zipcode']
lacity_zip_df.head()

In [ ]:
conn = sq.connect("testReferral.db")
pd_sql.to_sql(lacity_zip_df, 't_LAzip', conn, if_exists='replace', index=False)
conn.close()

In [ ]:
# Join DCFS and LA City by zip code to obtain LA City referrals only

conn = sq.connect("testReferral.db")
cur = conn.cursor()

query = '''
        SELECT *
        From t_dcfs
        JOIN t_LAzip
        ON t_dcfs.ref_zip = t_LAzip.zipcode
'''

ddf = pd.read_sql(query, conn) #183,048 rows matched

In [ ]:
ddf['ref_year'] = ddf['ref_date'].map(lambda x: x.split('-')[0])
ddf['ref_month'] = ddf['ref_date'].map(lambda x: x.split('-')[1])
ddf = ddf[ddf['ref_year']=="2016"]
ddf['count']=ddf['count'].astype('int')

In [ ]:
# #Load the data

# def loadSQL (tablename):
#     conn = sq.connect("testReferral.db")
#     cur = conn.cursor()
#     ddf = pd.read_sql('SELECT * FROM '+tablename, con=conn)
#     conn.close()
    
# loadSQL('t_dcfs')
# ddf.head

In [ ]:
ddf.head()

In [ ]:
grouped = ddf['count'].astype('int').groupby(ddf['referral_response_desc'])
grouped.sum()

In [ ]:
zip_ref_total = ddf.groupby('zipcode')['count'].sum()

In [ ]:
zip_ethnicity = ddf.groupby(['zipcode','ethnicity'])['count'].sum()/zip_ref_total*100

In [ ]:
zip_ethnicity

In [ ]:
zip_ethnicity.unstack().plot(kind='barh', stacked=True, fontsize=8)

### Crime records

In [36]:
client_crime = Socrata('data.lacity.org','7pTgt6f2oTY53aDI1jXNJoNZD')
offset_temp = 0
crime_df_1=pd.DataFrame()

while True:
    results = client_crime.get("7fvc-faax", limit=100000, offset=offset_temp)
    crime_df_temp = pd.DataFrame.from_records(results)
    if crime_df_temp.empty == True:
        break
    crime_df_1 = crime_df_1.append(crime_df_temp)
    offset_temp+=100000
    print(offset_temp)
    #time.sleep(0.1)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [186]:
crime_df = crime_df_1.copy()

In [201]:
temp = crime_df['location_1'].astype('str')

IndexError: list index out of range

In [223]:
crime_df['location_1'] = crime_df['location_1'].astype('str')
crime_df['long'] = crime_df['location_1'].map(lambda x: x.split(']')[0].split('[')[-1].split(',')[0])
crime_df['lat'] = crime_df['location_1'].map(lambda x: x.split(']')[0].split('[')[-1].split(',')[-1])
crime_df['year_rptd'] = crime_df['date_rptd'].map(lambda x: x.split('-')[0])
crime_df['month_rptd'] = crime_df['date_rptd'].map(lambda x: x.split('-')[1])
crime_df['count'] = 1
conn = sq.connect("testReferral.db")
cur = conn.cursor()
pd_sql.to_sql(crime_df, 't_crime', conn, if_exists='replace', index=False)
conn.close()

In [224]:
crime_df['year_rptd'].head()

0    2010
1    2010
2    2010
3    2010
4    2010
Name: year_rptd, dtype: object

In [3]:
conn = sq.connect("testReferral.db")
cur = conn.cursor()

query = '''
        SELECT date_rptd, month_rptd, crm_cd, crm_cd_desc,weapon_used_cd, count, lat, long
        FROM t_crime 
        WHERE year_rptd == 2016
        '''

cdf = pd.read_sql(query,con=conn)
conn.close()

In [4]:
len(cdf)

225696

In [5]:
from uszipcode import Zipcode
from uszipcode import SearchEngine

def zipcode_converter (lat, long):
    search = SearchEngine(simple_zipcode=True)
    result = search.by_coordinates(lat,long,radius=30,returns=1)
    return result[0].zipcode


In [9]:
zipcode_converter(34.0364,-118.2603)

'90015'

In [6]:
cdf.iloc[1,5].split('[')[1].split(']')[0].split(',')

['-118.2603', ' 34.0364']

In [239]:
float(cdf.iloc[i,6])

33.7547

In [6]:
cdf['zipcode'] = 99999
zip_list = list()
for i in range(len(cdf)):
    if i % 20000 == 0:
        print(i)
    try:
        lat = float(cdf.iloc[i,6])
        long = float(cdf.iloc[i,7])
        zipcode = zipcode_converter(lat, long)
        zip_list.append(zipcode)
    except Exception as e:
        print(e)
        zip_list.append('.')
        continue



0
20000
list index out of range
list index out of range
40000
list index out of range
60000
80000
list index out of range
list index out of range
list index out of range
list index out of range
100000
list index out of range
120000
list index out of range
list index out of range
140000
160000
list index out of range
list index out of range
list index out of range
180000
list index out of range
200000
list index out of range
list index out of range
list index out of range
220000
list index out of range
list index out of range
list index out of range


In [32]:
cdf['zips'] = zip_list

In [34]:
cdf = cdf.drop(['zipcode'], axis=1)

In [35]:
conn = sq.connect("testReferral.db")
cur = conn.cursor()
pd_sql.to_sql(cdf, 't_crime_zip', conn, if_exists='replace', index=False)
conn.close()

In [36]:
# Join Crime data and LA zipcode
conn = sq.connect("testReferral.db")
cur = conn.cursor()

query = '''
        SELECT *
        From t_crime_zip
        JOIN t_LAzip
        ON t_crime_zip.zips = t_LAzip.zipcode
'''

cdf_zip = pd.read_sql(query, conn) #224,509 matched of 225,696

In [37]:
cdf_zip.head()

,date_rptd,month_rptd,crm_cd,crm_cd_desc,weapon_used_cd,count,lat,long,zips,primary_key,zipcode
0,2016-01-30T00:00:00.000,01,626,INTIMATE PARTNER - SIMPLE ASSAULT,400,1,34.0478,-118.2502,90014,13,90014
1,2016-06-24T00:00:00.000,06,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",400,1,34.0364,-118.2603,90015,14,90015
2,2016-08-03T00:00:00.000,08,624,BATTERY - SIMPLE ASSAULT,400,1,34.0449,-118.2458,90014,13,90014
3,2016-09-09T00:00:00.000,09,815,SEXUAL PENETRATION W/FOREIGN OBJECT,400,1,34.0516,-118.2561,90071,59,90071
4,2016-10-08T00:00:00.000,10,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,511,1,34.0467,-118.252,90014,13,90014


In [41]:
cdf_grouped = cdf_zip.groupby(['zipcode','month_rptd'])['count'].sum()
cdf_grouped

zipcode  month_rptd
90001    01             82
         02             63
         03             86
         04             57
         05             89
         06             95
         07             84
         08             74
         09             87
         10             84
         11             63
         12             77
90002    01            275
         02            239
         03            248
         04            241
         05            279
         06            303
         07            305
         08            297
         09            266
         10            304
         11            259
         12            275
90003    01            551
         02            433
         03            512
         04            525
         05            537
         06            583
                      ... 
91606    07            260
         08            278
         09            242
         10            219
         11            235
        